# NanoPyx "Codeless" Jupyter Notebook

### This notebook allows you to generate a super-resolution image using SRRF (Super-Resolution Radiality Fluctuations) and then assess quality control metrics such as an Error Map (NanoJ-SQUIRREL), FRC (Fourier Ring Correlation) and Decorrelation analysis.
  
### To use this notebook you don't need to interact with any code, just run cells in order and a graphical user interface will pop-up showcasing the parameters for each step

### If you found this work useful for your research please consider citing: https://doi.org/10.5281/zenodo.7699280
  
**SRRF**: Culley S, Tosheva KL, Matos Pereira P, Henriques R. SRRF: Universal live-cell super-resolution microscopy. Int J Biochem Cell Biol. 2018 Aug;101:74-79. doi: 10.1016/j.biocel.2018.05.014. Epub 2018 May 28. PMID: 29852248; PMCID: PMC6025290.

**Error Map**: Culley, S., Albrecht, D., Jacobs, C. et al. Quantitative mapping and minimization of super-resolution optical imaging artifacts. Nat Methods 15, 263–266 (2018). https://doi.org/10.1038/nmeth.4605
  
**FRC**: Nieuwenhuizen RP, Lidke KA, Bates M, Puig DL, Grünwald D, Stallinga S, Rieger B. Measuring image resolution in optical nanoscopy. Nat Methods. 2013 Jun;10(6):557-62. doi: 10.1038/nmeth.2448. Epub 2013 Apr 28. PMID: 23624665; PMCID: PMC4149789.  
  
**DecorrAnalysis**: Descloux A, Grußmayer KS, Radenovic A. Parameter-free image resolution estimation based on decorrelation analysis. Nat Methods. 2019 Sep;16(9):918-924. doi: 10.1038/s41592-019-0515-7. Epub 2019 Aug 26. PMID: 31451766.  

## Notebook setup cell

In [ ]:
#@title Install NanoPyx, import necessary libraries and connect to Google Drive
!pip install "nanopyx[jupyter] @git+https://github.com/HenriquesLab/NanoPyx.git" --quiet
import io
import os
import sys
import skimage
import stackview
import numpy as np
import tifffile as tiff
import matplotlib as mpl
import ipywidgets as widgets
from IPython.display import display, clear_output
from matplotlib import pyplot as plt

from nanopyx.core.utils.easy_gui import EasyGui
from nanopyx.core.utils.find_files import find_files

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import output
    output.enable_custom_widget_manager()
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
#@title Load image stack
# Create a GUI
gui_data = EasyGui("Data Loader")

def on_button_load_data_clicked(b):
    clear_output()
    gui_data.show()
    global dataset_original
    # disable button
    gui_data["load_data"].disabled = True
    gui_data["load_data"].description = "Loading..."
    dataset_original = tiff.imread(gui_data["upload"].selected)
    gui_data["load_data"].disabled = False
    gui_data["load_data"].description = "Load data"
    display(stackview.slice(dataset_original, continuous_update=True,
    colormap=gui_data["cmaps"].value))

gui_data.add_label("Select data to use:")
gui_data.add_file_upload("upload")
gui_data.add_dropdown("cmaps", description="Colormap:",
                      options=sorted(list(mpl.colormaps)),
                      value="viridis", remember_value=True)
gui_data.add_button("load_data", description="Load data")
gui_data["load_data"].on_click(on_button_load_data_clicked)
gui_data.show()

## Now let's use SRRF to generate a super-resolution image

In [ ]:
#@title Create SRRF GUI
gui_srrf = EasyGui("srrf")
from nanopyx.methods.srrf import SRRF

def run_srrf(b):
    clear_output()
    gui_srrf.show()
    gui_srrf.save_settings()
    ring_radius = gui_srrf["ring_radius"].value
    magnification = gui_srrf["magnification"].value
    frames_per_timepoint = gui_srrf["frames_per_timepoint"].value
    srrf_order = gui_srrf["srrf_order"].value
    # disable button while running
    gui_srrf["run"].disabled = True
    gui_srrf["run"].description = "Running..."
    srrf = SRRF(magnification, ring_radius)
    global dataset_srrf
    dataset_srrf = srrf.calculate(dataset_original, frames_per_timepoint, srrf_order)
    # enable button again
    gui_srrf["run"].disabled = False
    gui_srrf["run"].description = "Run"
    if gui_srrf["save"].value:
        path = gui_data["upload"].selected_path
        name = gui_data["upload"].selected_filename.split(".")[0]
        tiff.imwrite(path + os.sep + name + "_srrf.tif", dataset_srrf[0])
    display(stackview.curtain(dataset_srrf[0], dataset_srrf[1],
                              colormap=gui_srrf["cmaps"].value,
                              curtain_colormap=gui_srrf["cmaps"].value,
                              continuous_update=True))

gui_srrf.add_float_slider("ring_radius", description="Ring Radius:", min=0.1, max=3.0, value=0.5, remember_value=True)
gui_srrf.add_int_slider("magnification", description="Magnification:", min=1, max=10, value=5)
gui_srrf.add_int_slider("srrf_order", description="SRRF order:", min=-1, max=4, value=3)
gui_srrf.add_label("-=-= Time-Lapse =-=-")
gui_srrf.add_int_slider("frames_per_timepoint", description="Frames per time-point (0 - auto)", min=1, max=dataset_original.shape[0], value=dataset_original.shape[0]//2)
gui_srrf.add_checkbox("save", description="Save Output", value=True)
gui_srrf.add_dropdown("cmaps", description="Colormap:", 
                      options=sorted(list(mpl.colormaps)),
                      value="viridis", remember_value=True)
gui_srrf.add_button("run", description="Run")
gui_srrf['run'].on_click(run_srrf)
gui_srrf.show()

## Let's use NanoPyx to generate an error map of the SRRF image

In [ ]:
#@title Create Error Map GUI
gui_error = EasyGui("Error")

import numpy as np
from matplotlib import pyplot as plt
from nanopyx.core.transform.new_error_map import ErrorMap

def run_error(b):
    clear_output()
    gui_error.show()
    gui_error.save_settings()
    gui_error["run"].disabled = True
    gui_error["run"].description = "Calculating..."
    global errormap
    error_map = ErrorMap()
    error_map.optimise(np.mean(dataset_original, axis=0), np.mean(dataset_srrf[0], axis=0))
    gui_error["run"].disabled = False
    gui_error["run"].description = "Calculate"
    print("RSE: ", error_map.getRSE())
    print("RSP: ", error_map.getRSP())
    errormap = np.array(error_map.imRSE)
    if gui_error["save"].value:
        path = gui_data["upload"].selected_path
        name = gui_data["upload"].selected_filename.split(".")[0]
        tiff.imwrite(path + os.sep + name + "_error_map.tif", errormap)
    plt.imshow(errormap)
    plt.axis("off")
    plt.show()
    
gui_error.add_checkbox("save", description="Save output", value=True)
gui_error.add_dropdown("cmaps", description="Colormap:",
                       options=sorted(list(mpl.colormaps)),
                       value="viridis", remember_value=True)
gui_error.add_button("run", description="Calculate")
gui_error["run"].on_click(run_error)
gui_error.show()

## Let's compare the resolution of the raw data with the SRRF using FRC and DecorrelationAnalysis. Let's start with calculation the FRC resolution of the raw data.

In [ ]:
#@title Create FRC GUI for original data
gui_frc_1 = EasyGui("FRC")

import numpy as np
from nanopyx.core.analysis.frc import FIRECalculator

def run_frc(b):
    clear_output()
    gui_frc_1.show()
    gui_frc_1.save_settings()
    pixel_size = gui_frc_1["pixel_size"].value
    units = gui_frc_1["units"].value
    first_frame = gui_frc_1["first_frame"].value
    second_frame = gui_frc_1["second_frame"].value
    gui_frc_1["run"].disabled = True
    gui_frc_1["run"].description = "Calculating..."
    global frc_calculator_raw
    frc_calculator_raw = FIRECalculator(pixel_size=pixel_size, units=units)
    frc_calculator_raw.calculate_fire_number(dataset_original[first_frame], dataset_original[second_frame])
    gui_frc_1["run"].disabled = False
    gui_frc_1["run"].description = "Calculate"
    plot = frc_calculator_raw.plot_frc_curve()
    if gui_frc_1["save"].value:
        path = gui_data["upload"].selected_path
        name = gui_data["upload"].selected_filename.split(".")[0]
        tiff.imwrite(path + os.sep + name + "_original_FRC.tif", plot)
    plt.imshow(plot)
    plt.axis("off")
    plt.show()
    
gui_frc_1.add_int_slider("pixel_size", description="Pixel Size:", min=0.01, max=1000, value=100, remember_value=True)
gui_frc_1.add_dropdown("units", description="Units: ", options=["nm", "um", "mm"], value="nm")
gui_frc_1.add_int_slider("first_frame", description="First Frame:", min=0, max=dataset_original[0].shape[0]-1, value=0)
gui_frc_1.add_int_slider ("second_frame", description="Second Frame:", min=0, max=dataset_original[0].shape[0]-1, value=1)
gui_frc_1.add_checkbox("save", description="Save Output", value=True)
gui_frc_1.add_button("run", description="Calculate")
gui_frc_1["run"].on_click(run_frc)
gui_frc_1.show()

## Now let's measure the resolution of the generated SRRF image using FRC

In [ ]:
#@title Create FRC GUI for SRRF data
gui_frc = EasyGui("FRC")

from nanopyx.core.analysis.frc import FIRECalculator

def run_frc(b):
    clear_output()
    gui_frc.show()
    gui_frc.save_settings()
    pixel_size = gui_frc["pixel_size"].value
    units = gui_frc["units"].value
    first_frame = gui_frc["first_frame"].value
    second_frame = gui_frc["second_frame"].value
    gui_frc["run"].disabled = True
    gui_frc["run"].description = "Calculating..."
    global frc_calculator
    frc_calculator = FIRECalculator(pixel_size=pixel_size, units=units)
    frc_calculator.calculate_fire_number(dataset_srrf[0][first_frame], dataset_srrf[0][second_frame])
    gui_frc["run"].disabled = False
    gui_frc["run"].description = "Calculate"
    plot = frc_calculator.plot_frc_curve()
    if gui_frc["save"].value:
        path = gui_data["upload"].selected_path
        name = gui_data["upload"].selected_filename.split(".")[0]
        tiff.imwrite(path + os.sep + name + "_SRRF_FRC.tif", plot)
    plt.imshow(plot)
    plt.axis("off")
    plt.show()
    
gui_frc.add_int_slider("pixel_size", description="Pixel Size:", min=0.01, max=1000, value=20, remember_value=True)
gui_frc.add_dropdown("units", description="Units: ", options=["nm", "um", "mm"], value="nm")
gui_frc.add_int_slider("first_frame", description="First Frame:", min=0, max=dataset_srrf[0].shape[0]-1, value=0)
gui_frc.add_int_slider ("second_frame", description="Second Frame:", min=0, max=dataset_srrf[0].shape[0]-1, value=1)
gui_frc.add_checkbox("save", description="Save Ouput", value=True)
gui_frc.add_button("run", description="Calculate")
gui_frc["run"].on_click(run_frc)
gui_frc.show()

## Let's do the same using Decorrelation Analysis, starting with the original data.

In [ ]:
#@title Create Decorr GUI for original data
gui_decorr_1 = EasyGui("DecorrAnalysis")

from nanopyx.core.analysis.decorr import DecorrAnalysis

def run_decorr(b):
    clear_output()
    gui_decorr_1.show()
    gui_decorr_1.save_settings()
    pixel_size = gui_decorr_1["pixel_size"].value
    units = gui_decorr_1["units"].value
    rmin = gui_decorr_1["rmin"].value
    rmax = gui_decorr_1["rmax"].value
    gui_decorr_1["run"].disabled = True
    gui_decorr_1["run"].description = "Calculating..."
    global decorr_calculator_raw
    decorr_calculator_raw = DecorrAnalysis(pixel_size=pixel_size, units=units, rmin=rmin, rmax=rmax)
    decorr_calculator_raw.run_analysis(np.mean(dataset_original, axis=0))
    gui_decorr_1["run"].disabled = False
    gui_decorr_1["run"].description = "Calculate"
    plot = decorr_calculator_raw.plot_results()
    if gui_decorr_1["save"].value:
        path = gui_data["upload"].selected_path
        name = gui_data["upload"].selected_filename.split(".")[0]
        tiff.imwrite(path + os.sep + name + "_original_decorr_analysis.tif", plot)
    plt.imshow(plot)
    plt.axis("off")
    plt.show()
    
gui_decorr_1.add_int_slider("pixel_size", description="Pixel Size:", min=0.01, max=1000, value=100, remember_value=True)
gui_decorr_1.add_dropdown("units", description="Units: ", options=["nm", "um", "mm"], value="nm")
gui_decorr_1.add_float_slider("rmin", description="Radius Min:", min=0.0, max=0.5, value=0.0)
gui_decorr_1.add_float_slider("rmax", description="Radius Max:", min=0.5, max=1.0, value=1.0)
gui_decorr_1.add_checkbox("save", description="Save Output", value=True)
gui_decorr_1.add_button("run", description="Calculate")
gui_decorr_1["run"].on_click(run_decorr)
gui_decorr_1.show()

## Now let's measure the resolution of the generated SRRF image using DecorrelationAnalysis

In [ ]:
#@title Create Decorr GUI for SRRF data
gui_decorr = EasyGui("DecorrAnalysis")

from nanopyx.core.analysis.decorr import DecorrAnalysis

def run_decorr(b):
    clear_output()
    gui_decorr.show()
    gui_decorr.save_settings()
    pixel_size = gui_decorr["pixel_size"].value
    units = gui_decorr["units"].value
    first_frame = gui_decorr["first_frame"].value
    rmin = gui_decorr["rmin"].value
    rmax = gui_decorr["rmax"].value
    gui_decorr["run"].disabled = True
    gui_decorr["run"].description = "Calculating..."
    global decorr_calculator
    decorr_calculator = DecorrAnalysis(pixel_size=pixel_size, units=units, rmin=rmin, rmax=rmax)
    decorr_calculator.run_analysis(dataset_srrf[0][first_frame])
    gui_decorr["run"].disabled = False
    gui_decorr["run"].description = "Calculate"
    plot = decorr_calculator.plot_results()
    if gui_decorr["save"].value:
        path = gui_data["upload"].selected_path
        name = gui_data["upload"].selected_filename.split(".")[0]
        tiff.imwrite(path + os.sep + name + "_SRRF_decorr_analysis.tif", plot)
    plt.imshow(plot)
    plt.axis("off")
    plt.show()
    
gui_decorr.add_int_slider("pixel_size", description="Pixel Size:", min=0.01, max=1000, value=100, remember_value=True)
gui_decorr.add_dropdown("units", description="Units: ", options=["nm", "um", "mm"], value="nm")
gui_decorr.add_int_slider("first_frame", description="Frame to be used:", min=0, max=dataset_srrf[0].shape[0]-1, value=0)
gui_decorr.add_float_slider("rmin", description="Radius Min:", min=0.0, max=0.5, value=0.0)
gui_decorr.add_float_slider("rmax", desctiption="Radius Max:", min=0.5, max=1.0, value=1.0)
gui_decorr.add_checkbox("save", description="Save Output", value=True)
gui_decorr.add_button("run", description="Calculate")
gui_decorr["run"].on_click(run_decorr)
gui_decorr.show()